__собираем предложения по квартирам__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [3]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/flat/avito' 
cian_raw_data_path = 'data/raw/flat/cian'

result_file_path = 'data/data_flat.pkl'

In [4]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [5]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/flat/avito/avito_2022-08-26_17-29_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-29_13-16_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-31_17-42_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-01_14-07_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-02_13-04_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-05_14-02_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-06_13-09_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-07_12-39_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-08_13-36_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-12_13-44_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-13_13-55_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-15_10-27_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-16_15-07_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-19_12-36_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-22_09-48_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-26_11-20_raw.xlsx',
 'data/raw/flat/avito/avito_2022-10-08_20-34_raw.xlsx',
 'data/raw/flat/avito/avito_2022-10-14_15-48_raw

In [6]:
# data_avito = pd.read_excel('data/raw/flat/avito/avito_2023-03-29_15-32_raw.xlsx')
# display(len(data_avito))

In [7]:
data_avito = pd.concat([
        pd.read_excel(f)
        # .dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

display(len(data_avito))

90782

In [8]:
data_avito = data_avito[ ~data_avito['adr'].isnull() ] 
display(len(data_avito))

84458

In [9]:
# data_avito.isnull().any()
# data_avito[data_avito['adr'].isnull() ] 

In [10]:
from lib.avito import AvitoDataCleanerRealtyFlat

data_avito = AvitoDataCleanerRealtyFlat().transform( data_avito )

print(len(data_avito))

84458


In [11]:
data_avito['dt'] = data_avito['ts'].dt.date
data_avito.groupby(['dt'])['avito_id'].count()

dt
2022-08-26    3487
2022-08-29    3458
2022-08-31    3420
2022-09-01    3431
2022-09-02    3519
2022-09-05    3476
2022-09-06    3428
2022-09-07    3115
2022-09-08    3270
2022-09-12    3106
2022-09-13    3413
2022-09-15    3532
2022-09-16    3391
2022-09-19    3555
2022-09-22    3631
2022-09-26    3576
2022-10-08    3599
2022-10-14    3474
2022-11-01    3144
2022-11-04    3144
2022-11-07    3257
2022-11-15    3218
2022-11-28    3596
2023-01-10    3038
2023-03-29    3180
Name: avito_id, dtype: int64

In [12]:
assert len(data_avito)>0

In [13]:
# data_avito.info()

In [14]:
# data_avito.isnull().any()

In [15]:
# data_avito.query('nrooms==0')
# data_avito[ data_avito['title'].str.len()<1]

## собираем данные циан.ру

In [16]:
import re
from os import listdir

raw_data_files = sorted([
        cian_raw_data_path+'/'+f 
        for f in listdir(cian_raw_data_path) 
        if re.match(r'.+\.pkl$',f)
        #if re.match(r'.+\.xlsx$',f)
    
    ])
raw_data_files

['data/raw/flat/cian/cian_2023-01-10_11-25_raw.pkl',
 'data/raw/flat/cian/cian_2023-01-27_16-37_raw.pkl',
 'data/raw/flat/cian/cian_2023-03-09_12-06_raw.pkl',
 'data/raw/flat/cian/cian_2023-03-29_15-28_raw.pkl']

In [17]:
data_cian = pd.concat([
        pd.read_pickle(f)
         .dropna()
         .drop_duplicates(['LinkArea','Description',])
         .rename(columns={'page':'cian_page'})
         .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

In [18]:
data_cian.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1988 entries, 0 to 1987
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OfferTitle     1988 non-null   object        
 1   OfferSubtitle  1988 non-null   object        
 2   Deadline       1988 non-null   object        
 3   MainPrice      1988 non-null   object        
 4   PriceInfo      1988 non-null   object        
 5   GeoLabel       1988 non-null   object        
 6   TimeLabel      1988 non-null   object        
 7   LinkArea       1988 non-null   object        
 8   Description    1988 non-null   object        
 9   cian_page      1988 non-null   int64         
 10  ts             1988 non-null   datetime64[ns]
 11  place          1988 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 186.5+ KB


In [19]:
data_cian.isnull().any()

OfferTitle       False
OfferSubtitle    False
Deadline         False
MainPrice        False
PriceInfo        False
GeoLabel         False
TimeLabel        False
LinkArea         False
Description      False
cian_page        False
ts               False
place            False
dtype: bool

In [20]:
swap_title = data_cian['OfferSubtitle'].str.match(r'.*\d+ м², \d+/\d+ эт.*') 

data_cian = pd.concat([
        data_cian[ ~swap_title ],
        data_cian[ swap_title ]
            .rename(columns={'OfferTitle':'OfferSubtitle','OfferSubtitle':'OfferTitle'})
    ]).reset_index(drop=True)


In [21]:
# data_cian

In [22]:
from lib.cian import CianDataCleaner

data_cian = CianDataCleaner().transform( data_cian )

print(len(data_cian))

1988


In [23]:
assert len(data_cian)>0

In [24]:
data_cian.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1988 entries, 0 to 1987
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          1988 non-null   object        
 1   obj_name       1988 non-null   object        
 2   adr            1988 non-null   object        
 3   nrooms         1988 non-null   int64         
 4   floor          1988 non-null   int64         
 5   nfloors        1988 non-null   int64         
 6   area           1988 non-null   float64       
 7   is_studio      1988 non-null   bool          
 8   is_apartment   1988 non-null   bool          
 9   is_part        1988 non-null   bool          
 10  is_auction     1988 non-null   bool          
 11  is_openspace   1988 non-null   bool          
 12  is_roof        1988 non-null   bool          
 13  is_SNT         1988 non-null   bool          
 14  price          1988 non-null   int64         
 15  priceM         1988 n

In [25]:
data_cian.isnull().any()

title            False
obj_name         False
adr              False
nrooms           False
floor            False
nfloors          False
area             False
is_studio        False
is_apartment     False
is_part          False
is_auction       False
is_openspace     False
is_roof          False
is_SNT           False
price            False
priceM           False
is_last_floor    False
cian_url         False
description      False
cian_page        False
ts               False
place            False
dtype: bool

In [26]:
# data_cian
# data_cian.query('nrooms==0')
# data_cian[ data_cian['title'].str.len()<1 ]
# data_cian[ data_cian['obj_name'].str.len()>0 ][['title','obj_name',]] # .to_csv('tmp/cian_title.csv',sep='\t')|

----

In [27]:
data_avito['url'] = 'https://avito.ru/' + data_avito['avito_id'].astype(int).astype(str)
data_avito = data_avito.drop(columns=['avito_id'])
data_cian = data_cian.rename(columns={'cian_url':'url'})

In [28]:
data = pd.concat([data_avito,data_cian]).reset_index(drop=True) 
print(len(data))
assert len(data)>0

86446


In [29]:
data.sample()

,title,price,adr,description,obj_name,ts,avito_page,place,nrooms,floor,...,is_part,is_auction,is_openspace,is_roof,is_SNT,priceM,is_last_floor,dt,url,cian_page
31616,"2-к. апартаменты, 41,7 м², 1/4 эт.",6700000,"Гагаринский муниципальный округ, микрорайон Казачья Бухта","Продам апартаменты с видом на море. Общая площадь 41.7 м. Кв., сделан качественный ремонт. Расположен на 1 эт.- 4 эт. Дома. Апартаменты сделаны в двух уровнях. На первом уровне: комната отдыха, коридор, санузел. На втором этаже: кухня, санузел, постирочная",NaN,2022-09-12 13:44:49.319,25.00,sevastopol,2,1,...,False,False,False,False,False,6.70,False,2022-09-12,https://avito.ru/2400645717,NaN


## обновляем таблицу адресов

In [30]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

In [31]:
places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [32]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

  0%|          | 0/86446 [00:00<?, ?it/s]

  0%|          | 0/86446 [00:00<?, ?it/s]

In [33]:
data[['adr_orig','adr']].sample(10)

,adr_orig,adr
47999,"улица Горпищенко, 143 к1","Севастополь, улица Горпищенко, 143 к1"
13164,"Новороссийская улица , 74","Севастополь,Новороссийская улица , 74"
29876,"улица Тараса Шевченко, 18","Севастополь, улица Тараса Шевченко, 18"
38299,"улица Военных Строителей, 14","Севастополь, улица Военных Строителей, 14"
18453,"улица Адмирала Фадеева, 18","Севастополь, улица Адмирала Фадеева, 18"
13199,"улица Тараса Шевченко, 56","Севастополь, улица Тараса Шевченко, 56"
34409,"улица Токарева, 7 этап, блок-секция 3","Севастополь, улица Токарева, 7 этап, блок-секция 3"
36060,"Кача, улица Авиаторов, 1Гк6","Севастополь, Кача, улица Авиаторов, 1Гк6"
51107,"улица Гоголя, 34А","Севастополь, улица Гоголя, 34А"
29165,"улица Гоголя, 63","Севастополь, улица Гоголя, 63"


In [34]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[0])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

if len(loc_undef)>0:
    # определяем координаты по строке адреса
    loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
    logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')
    loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 
    assert len(loc)>0
    loc.to_pickle(loc_file_path)

[INFO    ] 2023-03-29 16:19:08 | LocationUpdater: 9400 addresses in index
[INFO    ] 2023-03-29 16:19:08 | LocationUpdater: 9783 addresses total
[INFO    ] 2023-03-29 16:19:08 | LocationUpdater: 9400 addresses defined
[INFO    ] 2023-03-29 16:19:08 | LocationUpdater: 383 addresses undefined


  0%|          | 0/383 [00:00<?, ?it/s]

[INFO    ] 2023-03-29 16:23:04 | LocationUpdater: 383 new addresses found


In [35]:
print( len(loc) )
loc.sample(3)

9783


,latitude,longitude,adr
7948,44.739677,33.556253,"Севастополь, Нахимовский район, Качинский район , СОТ Южный, 232"
3541,44.564365,33.456271,"Севастополь,ул. Бертье-Делагарда, 1"
1992,44.588064,33.453701,"Севастополь, улица Астана Кесаева, 10"


## дополняем данные геометкой

In [36]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

In [37]:
data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

всего записей: 86446
записей без геометки: 0


In [38]:
assert len(data)>0
data.to_pickle(result_file_path)

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86446 entries, 0 to 86445
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          86446 non-null  object        
 1   price          86446 non-null  int64         
 2   adr_orig       86446 non-null  object        
 3   description    86360 non-null  object        
 4   obj_name       27153 non-null  object        
 5   ts             86446 non-null  datetime64[ns]
 6   avito_page     84458 non-null  float64       
 7   place          86446 non-null  object        
 8   nrooms         86446 non-null  int64         
 9   floor          86446 non-null  int64         
 10  nfloors        86446 non-null  int64         
 11  area           86446 non-null  float64       
 12  is_studio      86446 non-null  bool          
 13  is_apartment   86446 non-null  bool          
 14  is_part        86446 non-null  bool          
 15  is_auction     8644

In [40]:
data[['ts']].max()

ts   2023-03-29 16:04:54.059
dtype: datetime64[ns]

In [41]:
data[['ts']].groupby('ts').size()

ts
2022-08-26 17:29:00.000000    3487
2022-08-29 13:16:00.000000    3458
2022-08-31 17:42:00.000000    3420
2022-09-01 14:07:00.000000    3431
2022-09-02 13:04:00.000000    3519
2022-09-05 14:02:00.000000    3476
2022-09-06 13:09:00.000000    3428
2022-09-07 12:46:11.483000    3115
2022-09-08 13:36:05.789000    3270
2022-09-12 13:44:49.319000    3106
2022-09-13 13:55:03.079000    3413
2022-09-15 10:27:26.895000    3532
2022-09-16 15:07:31.632000    3391
2022-09-19 12:36:43.740000    3555
2022-09-22 09:48:34.368000    3631
2022-09-26 11:20:17.002000    3576
2022-10-08 20:34:54.992000    3599
2022-10-14 15:48:44.706000    3474
2022-11-01 13:51:36.763000    3144
2022-11-04 17:11:28.841000    3144
2022-11-07 10:43:26.010000    3257
2022-11-15 13:12:30.596000    3218
2022-11-28 13:20:28.620000    3596
2023-01-10 11:25:11.144238     465
2023-01-10 11:25:50.588000    3038
2023-01-27 16:37:11.689592    1467
2023-03-09 12:06:11.404170      28
2023-03-29 15:28:49.835295      28
2023-03-29 16:04: